In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h2>Import pacakges and data</h2>

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

in_ph_19_21_feb=pd.read_csv('/kaggle/input/nek-310-lecture-7/incoming_phone.csv')

<h2>Dataprep</h2>
<h3>We filter out time half hours during normal workdays, 8:00-16:30, Mon-Fri</h3>

In [ ]:
in_ph_19_21_feb=in_ph_19_21_feb[(in_ph_19_21_feb['Veckodag'].isin([0,1,2,3,4]))&(in_ph_19_21_feb['Intervall'].isin(['08:00-08:30',
         '08:30-09:00',
         '09:00-09:30',
         '09:30-10:00',
         '10:00-10:30',
         '10:30-11:00',
         '11:00-11:30',
         '11:30-12:00',
         '12:00-12:30',
         '12:30-13:00',
         '13:00-13:30',
         '13:30-14:00',
         '14:00-14:30',
         '14:30-15:00',
         '15:00-15:30',
         '15:30-16:00',
         '16:00-16:30']))]

In [ ]:
in_ph_19_21_feb.info()

<h3>We transform time strings to seconds</h3>

In [ ]:
import datetime as dt

def get_total_seconds(stringDHMS):
   if (len(stringDHMS)>8): 
       timedeltaObj = dt.timedelta(days=int(stringDHMS[:-9])) + dt.datetime.strptime(stringDHMS[-8:], "%H:%M:%S") - dt.datetime(1900,1,1) 
   else:
       timedeltaObj = dt.datetime.strptime(stringDHMS, "%H:%M:%S") - dt.datetime(1900,1,1)
    
   return  timedeltaObj.total_seconds()

In [ ]:
in_ph_19_21_feb['Väntetid_Besvarade medel sec'] = [get_total_seconds(x) for x in in_ph_19_21_feb['Väntetid_Besvarade medel']]

in_ph_19_21_feb['Väntetid_Tapp2 medel sec'] = [get_total_seconds(x) for x in in_ph_19_21_feb['Väntetid_Tapp2 medel']]

in_ph_19_21_feb['Väntetid_Besvarade max sec'] = [get_total_seconds(x) for x in in_ph_19_21_feb['Väntetid_Besvarade max']]

in_ph_19_21_feb['Ärendetid_Medel'] = [get_total_seconds(x) for x in in_ph_19_21_feb['Ärendetid_Medel']]

<h2>Create "Year, month" for month-based time-series plots</h2>

In [ ]:
in_ph_19_21_feb['Datum']=pd.to_datetime(in_ph_19_21_feb['Datum'])

in_ph_19_21_feb['År månad']=in_ph_19_21_feb['Datum'].dt.strftime('%Y%m')

<h3> Make numeric weekdays to strings </h3>

In [ ]:
conditions_vd=[in_ph_19_21_feb['Veckodag']==0,
               in_ph_19_21_feb['Veckodag']==1,
               in_ph_19_21_feb['Veckodag']==2,
               in_ph_19_21_feb['Veckodag']==3,
               in_ph_19_21_feb['Veckodag']==4,
               in_ph_19_21_feb['Veckodag']==5,
               in_ph_19_21_feb['Veckodag']==6]

choices_vd=['Måndag','Tisdag','Onsdag','Torsdag','Fredag','Lördag','Söndag']

in_ph_19_21_feb['Veckodag']=np.select(conditions_vd,choices_vd)

<h2>Service levels per day</h2>

In [ ]:
sns.set_theme()
plotdata_sn_mn = pd.DataFrame(in_ph_19_21_feb['Servicegrad (%)'].groupby(in_ph_19_21_feb['Veckodag']).mean())

plotdata_sn_se = pd.DataFrame(in_ph_19_21_feb['Servicegrad (%)'].groupby(in_ph_19_21_feb['Veckodag']).sem())

ordning=['Måndag','Tisdag','Onsdag','Torsdag','Fredag']

fig, ax = plt.subplots()
rects1 = ax.bar(['Måndag','Tisdag','Onsdag','Torsdag','Fredag'], plotdata_sn_mn['Servicegrad (%)'].loc[ordning], color='b')
plotline1, caplines1, barlinecols1 = ax.errorbar(
        ['Måndag','Tisdag','Onsdag','Torsdag','Fredag'], plotdata_sn_mn['Servicegrad (%)'].loc[ordning], yerr=plotdata_sn_se['Servicegrad (%)'].loc[ordning], lolims=True,
        capsize = 0, ls='None', color='k')

caplines1[0].set_marker('_')
caplines1[0].set_markersize(20)

plt.rcParams["figure.figsize"] = (15,10)
plt.title('Servicegrad (%) medel Telefon',fontsize=20)
plt.xticks(fontsize=16,rotation=-45)
plt.xlabel('Veckodag',fontsize=16)
plt.yticks(fontsize=16)
plt.show()

<h3>Service level per half hour</h3>

In [ ]:
plotdata_sn_mn = pd.DataFrame(in_ph_19_21_feb['Servicegrad (%)'].groupby(in_ph_19_21_feb['Intervall']).mean()).reset_index()

plotdata_sn_se = pd.DataFrame(in_ph_19_21_feb['Servicegrad (%)'].groupby(in_ph_19_21_feb['Intervall']).sem()).reset_index()

fig, ax = plt.subplots()
rects1 = ax.bar(plotdata_sn_mn['Intervall'], plotdata_sn_mn['Servicegrad (%)'], color='b')
plotline1, caplines1, barlinecols1 = ax.errorbar(
        plotdata_sn_mn['Intervall'], plotdata_sn_mn['Servicegrad (%)'], yerr=plotdata_sn_se['Servicegrad (%)'], lolims=True,
        capsize = 0, ls='None', color='k')

for label in ax.xaxis.get_ticklabels():
    label.set_horizontalalignment('left')

caplines1[0].set_marker('_')
caplines1[0].set_markersize(20)

plt.rcParams["figure.figsize"] = (15,10)
plt.title('Servicegrad (%) medel Telefon',fontsize=20)
plt.xticks(fontsize=16,rotation=-45)
plt.xlabel('Tidsintervall',fontsize=16)
plt.yticks(fontsize=16)
plt.show()

<h3>Service level during the pandemic</h3>

In [ ]:
plotdata_sn_mn_c = pd.DataFrame(in_ph_19_21_feb[in_ph_19_21_feb['År månad'].isin(['202005','202006','202007','202008','202009','202010','202011','202012','202101','202102'])]['Servicegrad (%)'].groupby(in_ph_19_21_feb['Intervall']).mean()).reset_index()

plotdata_sn_se_c = pd.DataFrame(in_ph_19_21_feb[in_ph_19_21_feb['År månad'].isin(['202005','202006','202007','202008','202009','202010','202011','202012','202101','202102'])]['Servicegrad (%)'].groupby(in_ph_19_21_feb['Intervall']).sem()).reset_index()


fig, ax = plt.subplots()
rects1 = ax.bar(plotdata_sn_mn_c['Intervall'], plotdata_sn_mn_c['Servicegrad (%)'], color='r')
plotline1, caplines1, barlinecols1 = ax.errorbar(
        plotdata_sn_mn_c['Intervall'], plotdata_sn_mn_c['Servicegrad (%)'], yerr=plotdata_sn_se_c['Servicegrad (%)'], lolims=True,
        capsize = 0, ls='None', color='k')

for label in ax.xaxis.get_ticklabels():
    label.set_horizontalalignment('left')

caplines1[0].set_marker('_')
caplines1[0].set_markersize(20)


plt.rcParams["figure.figsize"] = (15,10)
plt.title('Servicegrad (%) medel Telefon under Corona',fontsize=20)
plt.xticks(fontsize=16,rotation=-45)
plt.xlabel('Tidsintervall',fontsize=16)
plt.yticks(fontsize=16)
plt.show()

<h3>Service level pre Corona</h3>

In [ ]:
plotdata_sn_mn_pc = pd.DataFrame(in_ph_19_21_feb[~in_ph_19_21_feb['År månad'].isin(['202005','202006','202007','202008','202009','202010','202011','202012','202101','202102'])]['Servicegrad (%)'].groupby(in_ph_19_21_feb['Intervall']).mean()).reset_index()

plotdata_sn_se_pc = pd.DataFrame(in_ph_19_21_feb[~in_ph_19_21_feb['År månad'].isin(['202005','202006','202007','202008','202009','202010','202011','202012','202101','202102'])]['Servicegrad (%)'].groupby(in_ph_19_21_feb['Intervall']).sem()).reset_index()


fig, ax = plt.subplots()

rects2 = ax.bar(plotdata_sn_mn_pc['Intervall'], plotdata_sn_mn_pc['Servicegrad (%)'], color='b')
plotline2, caplines2, barlinecols2 = ax.errorbar(
        plotdata_sn_mn_pc['Intervall'], plotdata_sn_mn_pc['Servicegrad (%)'], yerr=plotdata_sn_se_pc['Servicegrad (%)'], lolims=True,
        capsize = 0, ls='None', color='k')


for label in ax.xaxis.get_ticklabels():
    label.set_horizontalalignment('left')

caplines2[0].set_marker('_')
caplines2[0].set_markersize(20)

plt.rcParams["figure.figsize"] = (15,10)
plt.title('Servicegrad (%) medel Telefon före Corona',fontsize=20)
plt.xticks(fontsize=16,rotation=-45)
plt.xlabel('Tidsintervall',fontsize=16)
plt.yticks(fontsize=16)
plt.show()

<h3>Incoming calls</h3>

In [ ]:
plotdata_sn_mn = pd.DataFrame(in_ph_19_21_feb['Inkommande_Total'].groupby(in_ph_19_21_feb['Veckodag']).mean())

plotdata_sn_se = pd.DataFrame(in_ph_19_21_feb['Inkommande_Total'].groupby(in_ph_19_21_feb['Veckodag']).sem())

ordning=['Måndag','Tisdag','Onsdag','Torsdag','Fredag']

fig, ax = plt.subplots()
rects1 = ax.bar(['Måndag','Tisdag','Onsdag','Torsdag','Fredag'], plotdata_sn_mn['Inkommande_Total'].loc[ordning], color='b')
plotline1, caplines1, barlinecols1 = ax.errorbar(
        ['Måndag','Tisdag','Onsdag','Torsdag','Fredag'], plotdata_sn_mn['Inkommande_Total'].loc[ordning], yerr=plotdata_sn_se['Inkommande_Total'].loc[ordning], lolims=True,
        capsize = 0, ls='None', color='k')

caplines1[0].set_marker('_')
caplines1[0].set_markersize(20)

plt.rcParams["figure.figsize"] = (15,10)
plt.title('Inkommande samtal medel Telefon',fontsize=20)
plt.xticks(fontsize=16,rotation=-45)
plt.xlabel('Veckodag',fontsize=16)
plt.yticks(fontsize=16)
plt.show()

In [ ]:
plotdata_sn_mn = pd.DataFrame(in_ph_19_21_feb['Inkommande_Total'].groupby(in_ph_19_21_feb['Intervall']).mean()).reset_index()

plotdata_sn_se = pd.DataFrame(in_ph_19_21_feb['Inkommande_Total'].groupby(in_ph_19_21_feb['Intervall']).sem()).reset_index()

fig, ax = plt.subplots()
rects1 = ax.bar(plotdata_sn_mn['Intervall'], plotdata_sn_mn['Inkommande_Total'], color='b')
plotline1, caplines1, barlinecols1 = ax.errorbar(
        plotdata_sn_mn['Intervall'], plotdata_sn_mn['Inkommande_Total'], yerr=plotdata_sn_se['Inkommande_Total'], lolims=True,
        capsize = 0, ls='None', color='k')

for label in ax.xaxis.get_ticklabels():
    label.set_horizontalalignment('left')

caplines1[0].set_marker('_')
caplines1[0].set_markersize(20)

plt.rcParams["figure.figsize"] = (15,10)
plt.title('Inkommande samtal medel Telefon',fontsize=20)
plt.xticks(fontsize=16,rotation=-45)
plt.xlabel('Tidsintervall',fontsize=16)
plt.yticks(fontsize=16)
plt.show()

In [ ]:
plotdata_sn_mn_c = pd.DataFrame(in_ph_19_21_feb[in_ph_19_21_feb['År månad'].isin(['202005','202006','202007','202008','202009','202010','202011','202012','202101','202102'])]['Inkommande_Total'].groupby(in_ph_19_21_feb['Intervall']).mean()).reset_index()

plotdata_sn_se_c = pd.DataFrame(in_ph_19_21_feb[in_ph_19_21_feb['År månad'].isin(['202005','202006','202007','202008','202009','202010','202011','202012','202101','202102'])]['Inkommande_Total'].groupby(in_ph_19_21_feb['Intervall']).sem()).reset_index()


fig, ax = plt.subplots()
rects1 = ax.bar(plotdata_sn_mn_c['Intervall'], plotdata_sn_mn_c['Inkommande_Total'], color='r')
plotline1, caplines1, barlinecols1 = ax.errorbar(
        plotdata_sn_mn_c['Intervall'], plotdata_sn_mn_c['Inkommande_Total'], yerr=plotdata_sn_se_c['Inkommande_Total'], lolims=True,
        capsize = 0, ls='None', color='k')

for label in ax.xaxis.get_ticklabels():
    label.set_horizontalalignment('left')

caplines1[0].set_marker('_')
caplines1[0].set_markersize(20)


plt.rcParams["figure.figsize"] = (15,10)
plt.title('Inkommande total medel Telefon under Corona',fontsize=20)
plt.xticks(fontsize=16,rotation=-45)
plt.xlabel('Tidsintervall',fontsize=16)
plt.yticks(fontsize=16)
plt.show()

In [ ]:
plotdata_sn_mn_c = pd.DataFrame(in_ph_19_21_feb[~in_ph_19_21_feb['År månad'].isin(['202005','202006','202007','202008','202009','202010','202011','202012','202101','202102'])]['Inkommande_Total'].groupby(in_ph_19_21_feb['Intervall']).mean()).reset_index()

plotdata_sn_se_c = pd.DataFrame(in_ph_19_21_feb[~in_ph_19_21_feb['År månad'].isin(['202005','202006','202007','202008','202009','202010','202011','202012','202101','202102'])]['Inkommande_Total'].groupby(in_ph_19_21_feb['Intervall']).sem()).reset_index()


fig, ax = plt.subplots()
rects1 = ax.bar(plotdata_sn_mn_c['Intervall'], plotdata_sn_mn_c['Inkommande_Total'], color='b')
plotline1, caplines1, barlinecols1 = ax.errorbar(
        plotdata_sn_mn_c['Intervall'], plotdata_sn_mn_c['Inkommande_Total'], yerr=plotdata_sn_se_c['Inkommande_Total'], lolims=True,
        capsize = 0, ls='None', color='k')

for label in ax.xaxis.get_ticklabels():
    label.set_horizontalalignment('left')

caplines1[0].set_marker('_')
caplines1[0].set_markersize(20)


plt.rcParams["figure.figsize"] = (15,10)
plt.title('Inkommande total medel Telefon före Corona',fontsize=20)
plt.xticks(fontsize=16,rotation=-45)
plt.xlabel('Tidsintervall',fontsize=16)
plt.yticks(fontsize=16)
plt.show()

<h3>Incoming answered</h3>

In [ ]:


plotdata_sn_mn = pd.DataFrame(in_ph_19_21_feb['Inkommande_Besvarade'].groupby(in_ph_19_21_feb['Veckodag']).mean())

plotdata_sn_se = pd.DataFrame(in_ph_19_21_feb['Inkommande_Besvarade'].groupby(in_ph_19_21_feb['Veckodag']).sem())

ordning=['Måndag','Tisdag','Onsdag','Torsdag','Fredag']

fig, ax = plt.subplots()
rects1 = ax.bar(['Måndag','Tisdag','Onsdag','Torsdag','Fredag'], plotdata_sn_mn['Inkommande_Besvarade'].loc[ordning], color='b')
plotline1, caplines1, barlinecols1 = ax.errorbar(
        ['Måndag','Tisdag','Onsdag','Torsdag','Fredag'], plotdata_sn_mn['Inkommande_Besvarade'].loc[ordning], yerr=plotdata_sn_se['Inkommande_Besvarade'].loc[ordning], lolims=True,
        capsize = 0, ls='None', color='k')

caplines1[0].set_marker('_')
caplines1[0].set_markersize(20)

plt.rcParams["figure.figsize"] = (15,10)
plt.title('Inkommande besvarade medel Telefon',fontsize=20)
plt.xticks(fontsize=16,rotation=-45)
plt.xlabel('Veckodag',fontsize=16)
plt.yticks(fontsize=16)
plt.show()

<h3>Put on hold</h3>

In [ ]:


plotdata_sn_mn = pd.DataFrame(in_ph_19_21_feb['Inkommande_Tapp1'].groupby(in_ph_19_21_feb['Veckodag']).mean())

plotdata_sn_se = pd.DataFrame(in_ph_19_21_feb['Inkommande_Tapp1'].groupby(in_ph_19_21_feb['Veckodag']).sem())

ordning=['Måndag','Tisdag','Onsdag','Torsdag','Fredag']

fig, ax = plt.subplots()
rects1 = ax.bar(['Måndag','Tisdag','Onsdag','Torsdag','Fredag'], plotdata_sn_mn['Inkommande_Tapp1'].loc[ordning], color='b')
plotline1, caplines1, barlinecols1 = ax.errorbar(
        ['Måndag','Tisdag','Onsdag','Torsdag','Fredag'], plotdata_sn_mn['Inkommande_Tapp1'].loc[ordning], yerr=plotdata_sn_se['Inkommande_Tapp1'].loc[ordning], lolims=True,
        capsize = 0, ls='None', color='k')

caplines1[0].set_marker('_')
caplines1[0].set_markersize(20)

plt.rcParams["figure.figsize"] = (15,10)
plt.title('Inkommande Tapp 1 medel Telefon',fontsize=20)
plt.xticks(fontsize=16,rotation=-45)
plt.xlabel('Veckodag',fontsize=16)
plt.yticks(fontsize=16)
plt.show()

<h3>Hanging up</h3>

In [ ]:

plotdata_sn_mn = pd.DataFrame(in_ph_19_21_feb['Inkommande_Tapp2'].groupby(in_ph_19_21_feb['Veckodag']).mean())

plotdata_sn_se = pd.DataFrame(in_ph_19_21_feb['Inkommande_Tapp2'].groupby(in_ph_19_21_feb['Veckodag']).sem())

ordning=['Måndag','Tisdag','Onsdag','Torsdag','Fredag']

fig, ax = plt.subplots()
rects1 = ax.bar(['Måndag','Tisdag','Onsdag','Torsdag','Fredag'], plotdata_sn_mn['Inkommande_Tapp2'].loc[ordning], color='b')
plotline1, caplines1, barlinecols1 = ax.errorbar(
        ['Måndag','Tisdag','Onsdag','Torsdag','Fredag'], plotdata_sn_mn['Inkommande_Tapp2'].loc[ordning], yerr=plotdata_sn_se['Inkommande_Tapp2'].loc[ordning], lolims=True,
        capsize = 0, ls='None', color='k')

caplines1[0].set_marker('_')
caplines1[0].set_markersize(20)

plt.rcParams["figure.figsize"] = (15,10)
plt.title('Inkommande Tapp 2 medel Telefon',fontsize=20)
plt.xticks(fontsize=16,rotation=-45)
plt.xlabel('Veckodag',fontsize=16)
plt.yticks(fontsize=16)
plt.show()

In [ ]:
plotdata_sn_mn = pd.DataFrame(in_ph_19_21_feb['Inkommande_Tapp2'].groupby(in_ph_19_21_feb['Intervall']).mean()).reset_index()

plotdata_sn_se = pd.DataFrame(in_ph_19_21_feb['Inkommande_Tapp2'].groupby(in_ph_19_21_feb['Intervall']).sem()).reset_index()

fig, ax = plt.subplots()
rects1 = ax.bar(plotdata_sn_mn['Intervall'], plotdata_sn_mn['Inkommande_Tapp2'], color='b')
plotline1, caplines1, barlinecols1 = ax.errorbar(
        plotdata_sn_mn['Intervall'], plotdata_sn_mn['Inkommande_Tapp2'], yerr=plotdata_sn_se['Inkommande_Tapp2'], lolims=True,
        capsize = 0, ls='None', color='k')

for label in ax.xaxis.get_ticklabels():
    label.set_horizontalalignment('left')

caplines1[0].set_marker('_')
caplines1[0].set_markersize(20)

plt.rcParams["figure.figsize"] = (15,10)
plt.title('Inkommande Tapp 2 medel Telefon',fontsize=20)
plt.xticks(fontsize=16,rotation=-45)
plt.xlabel('Tidsintervall',fontsize=16)
plt.yticks(fontsize=16)
plt.show()

<h3>Hanging up during the pandemic</h3>

In [ ]:
plotdata_sn_mn = pd.DataFrame(in_ph_19_21_feb[in_ph_19_21_feb['År månad'].isin(['202005','202006','202007','202008','202009','202010','202011','202012','202101','202102'])]['Inkommande_Tapp2'].groupby(in_ph_19_21_feb['Intervall']).mean()).reset_index()

plotdata_sn_se = pd.DataFrame(in_ph_19_21_feb[in_ph_19_21_feb['År månad'].isin(['202005','202006','202007','202008','202009','202010','202011','202012','202101','202102'])]['Inkommande_Tapp2'].groupby(in_ph_19_21_feb['Intervall']).sem()).reset_index()

fig, ax = plt.subplots()
rects1 = ax.bar(plotdata_sn_mn['Intervall'], plotdata_sn_mn['Inkommande_Tapp2'], color='r')
plotline1, caplines1, barlinecols1 = ax.errorbar(
        plotdata_sn_mn['Intervall'], plotdata_sn_mn['Inkommande_Tapp2'], yerr=plotdata_sn_se['Inkommande_Tapp2'], lolims=True,
        capsize = 0, ls='None', color='k')

for label in ax.xaxis.get_ticklabels():
    label.set_horizontalalignment('left')

caplines1[0].set_marker('_')
caplines1[0].set_markersize(20)

plt.rcParams["figure.figsize"] = (15,10)
plt.title('Inkommande Tapp 2 medel Telefon under Corona',fontsize=20)
plt.xticks(fontsize=16,rotation=-45)
plt.xlabel('Tidsintervall',fontsize=16)
plt.yticks(fontsize=16)
plt.show()

<h3>Hanging up pre-pandemic</h3>

In [ ]:
plotdata_sn_mn = pd.DataFrame(in_ph_19_21_feb[~in_ph_19_21_feb['År månad'].isin(['202005','202006','202007','202008','202009','202010','202011','202012','202101','202102'])]['Inkommande_Tapp2'].groupby(in_ph_19_21_feb['Intervall']).mean()).reset_index()

plotdata_sn_se = pd.DataFrame(in_ph_19_21_feb[~in_ph_19_21_feb['År månad'].isin(['202005','202006','202007','202008','202009','202010','202011','202012','202101','202102'])]['Inkommande_Tapp2'].groupby(in_ph_19_21_feb['Intervall']).sem()).reset_index()

fig, ax = plt.subplots()
rects1 = ax.bar(plotdata_sn_mn['Intervall'], plotdata_sn_mn['Inkommande_Tapp2'], color='b')
plotline1, caplines1, barlinecols1 = ax.errorbar(
        plotdata_sn_mn['Intervall'], plotdata_sn_mn['Inkommande_Tapp2'], yerr=plotdata_sn_se['Inkommande_Tapp2'], lolims=True,
        capsize = 0, ls='None', color='k')

for label in ax.xaxis.get_ticklabels():
    label.set_horizontalalignment('left')

caplines1[0].set_marker('_')
caplines1[0].set_markersize(20)

plt.rcParams["figure.figsize"] = (15,10)
plt.title('Inkommande Tapp 2 medel Telefon före Corona',fontsize=20)
plt.xticks(fontsize=16,rotation=-45)
plt.xlabel('Tidsintervall',fontsize=16)
plt.yticks(fontsize=16)
plt.show()

<h3>Pandemic and pre in the same chart</h3>

In [ ]:
plotdata_sn_mn_pc = pd.DataFrame(in_ph_19_21_feb[~in_ph_19_21_feb['År månad'].isin(['202005','202006','202007','202008','202009','202010','202011','202012','202101','202102'])]['Inkommande_Tapp2'].groupby(in_ph_19_21_feb['Intervall']).mean()).reset_index()

plotdata_sn_mn_pc['Period']='Före Corona'

plotdata_sn_se_pc = pd.DataFrame(in_ph_19_21_feb[~in_ph_19_21_feb['År månad'].isin(['202005','202006','202007','202008','202009','202010','202011','202012','202101','202102'])]['Inkommande_Tapp2'].groupby(in_ph_19_21_feb['Intervall']).sem()).reset_index()

plotdata_sn_se_pc['Period']='Före Corona'

plotdata_sn_mn_c = pd.DataFrame(in_ph_19_21_feb[in_ph_19_21_feb['År månad'].isin(['202005','202006','202007','202008','202009','202010','202011','202012','202101','202102'])]['Inkommande_Tapp2'].groupby(in_ph_19_21_feb['Intervall']).mean()).reset_index()

plotdata_sn_mn_c['Period']='Corona'

plotdata_sn_se_c = pd.DataFrame(in_ph_19_21_feb[in_ph_19_21_feb['År månad'].isin(['202005','202006','202007','202008','202009','202010','202011','202012','202101','202102'])]['Inkommande_Tapp2'].groupby(in_ph_19_21_feb['Intervall']).sem()).reset_index()

plotdata_sn_se_c['Period']='Corona'

plotdata_sn_mn=pd.concat([plotdata_sn_mn_pc, plotdata_sn_mn_c], ignore_index=True)

plotdata_sn_se=pd.concat([plotdata_sn_se_pc, plotdata_sn_se_c], ignore_index=True)

plotdata_sn_mn.pivot(index='Intervall',columns='Period',values='Inkommande_Tapp2').plot(kind='bar')

plt.rcParams["figure.figsize"] = (15,10)
plt.title('Inkommande Tapp 2 medel Telefon före och under Corona',fontsize=20)
plt.xticks(fontsize=16,rotation=-45)
plt.xlabel('Tidsintervall',fontsize=16)
plt.legend(fontsize=16)
plt.yticks(fontsize=16)
plt.show()

<h3>Waiting times answered calls</h3>

In [ ]:
plotdata_sn_mn = pd.DataFrame(in_ph_19_21_feb['Väntetid_Besvarade medel sec'].groupby(in_ph_19_21_feb['Intervall']).mean()).reset_index()

plotdata_sn_se = pd.DataFrame(in_ph_19_21_feb['Väntetid_Besvarade medel sec'].groupby(in_ph_19_21_feb['Intervall']).sem()).reset_index()

plotdata_sn_mn= plotdata_sn_mn[~plotdata_sn_mn['Intervall'].isin(['16:00-16:30'])]
    
plotdata_sn_se =plotdata_sn_se[~plotdata_sn_se['Intervall'].isin(['16:00-16:30'])]

fig, ax = plt.subplots()
rects1 = ax.bar(plotdata_sn_mn['Intervall'], plotdata_sn_mn['Väntetid_Besvarade medel sec'], color='b')
plotline1, caplines1, barlinecols1 = ax.errorbar(
        plotdata_sn_mn['Intervall'], plotdata_sn_mn['Väntetid_Besvarade medel sec'], yerr=plotdata_sn_se['Väntetid_Besvarade medel sec'], lolims=True,
        capsize = 0, ls='None', color='k')

for label in ax.xaxis.get_ticklabels():
    label.set_horizontalalignment('left')

caplines1[0].set_marker('_')
caplines1[0].set_markersize(20)

plt.rcParams["figure.figsize"] = (15,10)
plt.title('Väntetid besvarade medel (sek) Telefon',fontsize=20)
plt.xticks(fontsize=16,rotation=-45)
plt.xlabel('Tidsintervall',fontsize=16)
plt.yticks(fontsize=16)
plt.show()

<h3>Waiting times answered calls during the pandemic</h3>

In [ ]:
plotdata_sn_mn = pd.DataFrame(in_ph_19_21_feb[in_ph_19_21_feb['År månad'].isin(['202005','202006','202007','202008','202009','202010','202011','202012','202101','202102'])]['Väntetid_Besvarade medel sec'].groupby(in_ph_19_21_feb['Intervall']).mean()).reset_index()

plotdata_sn_se = pd.DataFrame(in_ph_19_21_feb[in_ph_19_21_feb['År månad'].isin(['202005','202006','202007','202008','202009','202010','202011','202012','202101','202102'])]['Väntetid_Besvarade medel sec'].groupby(in_ph_19_21_feb['Intervall']).sem()).reset_index()

plotdata_sn_mn= plotdata_sn_mn[~plotdata_sn_mn['Intervall'].isin(['16:00-16:30'])]
    
plotdata_sn_se =plotdata_sn_se[~plotdata_sn_se['Intervall'].isin(['16:00-16:30'])]

fig, ax = plt.subplots()
rects1 = ax.bar(plotdata_sn_mn['Intervall'], plotdata_sn_mn['Väntetid_Besvarade medel sec'], color='r')
plotline1, caplines1, barlinecols1 = ax.errorbar(
        plotdata_sn_mn['Intervall'], plotdata_sn_mn['Väntetid_Besvarade medel sec'], yerr=plotdata_sn_se['Väntetid_Besvarade medel sec'], lolims=True,
        capsize = 0, ls='None', color='k')

for label in ax.xaxis.get_ticklabels():
    label.set_horizontalalignment('left')

caplines1[0].set_marker('_')
caplines1[0].set_markersize(20)

plt.rcParams["figure.figsize"] = (15,10)
plt.title('Väntetid besvarade medel (sek) Telefon under Corona',fontsize=20)
plt.xticks(fontsize=16,rotation=-45)
plt.xlabel('Tidsintervall',fontsize=16)
plt.yticks(fontsize=16)
plt.show()

<h3>Waiting times answered calls before the pandemic</h3>

In [ ]:
plotdata_sn_mn = pd.DataFrame(in_ph_19_21_feb[~in_ph_19_21_feb['År månad'].isin(['202005','202006','202007','202008','202009','202010','202011','202012','202101','202102'])]['Väntetid_Besvarade medel sec'].groupby(in_ph_19_21_feb['Intervall']).mean()).reset_index()

plotdata_sn_se = pd.DataFrame(in_ph_19_21_feb[~in_ph_19_21_feb['År månad'].isin(['202005','202006','202007','202008','202009','202010','202011','202012','202101','202102'])]['Väntetid_Besvarade medel sec'].groupby(in_ph_19_21_feb['Intervall']).sem()).reset_index()

plotdata_sn_mn= plotdata_sn_mn[~plotdata_sn_mn['Intervall'].isin(['16:00-16:30'])]
    
plotdata_sn_se =plotdata_sn_se[~plotdata_sn_se['Intervall'].isin(['16:00-16:30'])]

fig, ax = plt.subplots()
rects1 = ax.bar(plotdata_sn_mn['Intervall'], plotdata_sn_mn['Väntetid_Besvarade medel sec'], color='b')
plotline1, caplines1, barlinecols1 = ax.errorbar(
        plotdata_sn_mn['Intervall'], plotdata_sn_mn['Väntetid_Besvarade medel sec'], yerr=plotdata_sn_se['Väntetid_Besvarade medel sec'], lolims=True,
        capsize = 0, ls='None', color='k')

for label in ax.xaxis.get_ticklabels():
    label.set_horizontalalignment('left')

caplines1[0].set_marker('_')
caplines1[0].set_markersize(20)

plt.rcParams["figure.figsize"] = (15,10)
plt.title('Väntetid besvarade medel (sek) Telefon före Corona',fontsize=20)
plt.xticks(fontsize=16,rotation=-45)
plt.xlabel('Tidsintervall',fontsize=16)
plt.yticks(fontsize=16)
plt.show()

<h3>Put on hold over and put on hold and hung up over time</h3>

In [ ]:
import seaborn as sns
sns.set_theme()
plt.rcParams["figure.figsize"] = (15,10)
plt.plot(in_ph_19_21_feb.groupby(['År månad'])['Inkommande_Tapp2'].mean(),label='Tapp 2',linewidth=4)
plt.plot(in_ph_19_21_feb.groupby(['År månad'])['Inkommande_Tapp1'].mean(),label='Tapp 1',linewidth=4)
#plt.plot(Alldata6b.groupby(['TID_T_NOM_AVSL_MTH'])['RES_ERROR'].mean(),label='VOLKSWAGEN', linewidth=4)
#plt.plot(Alldata6c.groupby(['TID_T_NOM_AVSL_MTH'])['RES_ERROR'].mean(),label='BMW, AUDI, MB', linewidth=4)
plt.title("Mean Tapp",fontsize=20)
plt.ylabel('Mean',fontsize=20)
plt.xlabel('Month',fontsize=20)
plt.legend(fontsize=16)
#xmarks=[i for i in inkommande_tot_gt_zero['År månad']]
plt.xticks(fontsize=16,rotation=-45)
#plt.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.yticks(np.arange(0,0.6,step=0.1),fontsize=16,rotation=45)
plt.show()

<h3>Total incoming over time</h3>

In [ ]:
import seaborn as sns
sns.set_theme()
plt.rcParams["figure.figsize"] = (15,10)
plt.plot(in_ph_19_21_feb.groupby(['År månad'])['Inkommande_Total'].mean(),label='Inkommande',linewidth=4)
#plt.plot(in_ph_19_21_feb.groupby(['År månad'])['Inkommande_Tapp1'].mean(),label='Tapp 1',linewidth=4)
#plt.plot(Alldata6b.groupby(['TID_T_NOM_AVSL_MTH'])['RES_ERROR'].mean(),label='VOLKSWAGEN', linewidth=4)
#plt.plot(Alldata6c.groupby(['TID_T_NOM_AVSL_MTH'])['RES_ERROR'].mean(),label='BMW, AUDI, MB', linewidth=4)
plt.title("Totalt inkommande",fontsize=20)
plt.ylabel('Medel',fontsize=20)
plt.xlabel('Månad',fontsize=20)
plt.legend(fontsize=16)
#xmarks=[i for i in inkommande_tot_gt_zero['År månad']]
plt.xticks(fontsize=16,rotation=-45)
#plt.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.yticks(np.arange(5,30,step=3),fontsize=16,rotation=45)
plt.show()